# 5) Application d'un modèle d'apprentissage non supervisé

Dans cette partie notre but était d'établir un modèle d'apprentissage non supervisé sur le contenu même du mail pour essayer de dégager des mots qui revenaient assez souvent parmis tous ces mails. 
Cela avait pour but de nous aider par la suite à établir des classes pour ensuite essayer d'entrainer des modèles pour réussir à classer ces mails dans les bonnes classes.

In [19]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [20]:
dataset = pd.read_csv('/Users/pierreperrin/Desktop/IG4/semestre7/Projet_DataScience/dataset_cleaned.csv',dtype={"content": str},low_memory=False)

On se focalise juste sur le contenu du mail donc on le charge à part dans un autre dataframe

In [21]:
mail_content_brut=dataset['content']

Nous allons tout d'abord nettoyer le contenu des mails : la plus petite unité d'un texte est le mot. Nous avons donc créer une fonction pour nettoyer le texte et le convertir en une simple séquence de mots en éliminant tout ce qui n'a pas d'importance dans la compréhension d'un texte (pour un ordinateur) comme par exemple la ponctuation, les lettres majuscules, les balises HTML si c'est un contenu "salle" comme un mail automatique par exemple. 

In [22]:

import re

def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """
    
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text

In [23]:
mail_content_cleaned = mail_content_brut

for i in range(len(mail_content_brut)):
    mail_content_cleaned[i] = clean_text(str(mail_content_brut[i]))

<ipython-input-23-940d37f807ac>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mail_content_cleaned[i] = clean_text(str(mail_content_brut[i]))


In [25]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(mail_content_cleaned)

On utilise ensuite le model d'apprentissage non supervisé Kmean pour définir des clusters suivant les mots présents dans le contenu des mails.

In [28]:
true_k = 3
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(max_iter=100, n_clusters=3, n_init=1)

In [29]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [30]:
for i in range(true_k):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :20]:
        print("%s" % terms[ind])

Cluster 0:
enron
com
2001
00
jeff
thanks
pm
subject
message
mail
time
know
sent
2000
10
meeting
11
attached
original
power
Cluster 1:
ect
hou
enron
ees
2000
na
corp
cc
pm
subject
forwarded
delainey
david
eric
09
sally
12
bass
02
11
Cluster 2:
20
01
09
enron
energy
power
california
business
said
company
management
new
ect
market
018
2001
global
com
state
risk


On arrive pas a dégager précisement de themes différents suivant les clusters mais cela nous informe sur les mots qui reviennent souvent dans les mails. 

Ainsi on a decidé par la suite de créer d'autres variables suplémentaires à notre dataset qui representeront les thèmes auxquels se rapprochent les mails. On rajoute donc les colonnes meetings, business trip, hobbie et on regarde si chaque mail correspond à un ou plusieur de ces thèmes.

on décide de garder les mots les plus pertinent pour créer des colonnes correspondant au thème du mail. Cela nous permettra d'établir des modèles de prédiction supervisés.